In [1]:
#importing packages
import random 
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def basicbandit(a):
    fi=math.pi/4
    w=1 #the reward if we won
    l=0 #the reward if we lose
    ## here we realize a simulation of the experiment
    b=random.random() # choosing some random uniformely distributed number between 0 and 1
    if b<0.5*(1+math.cos(fi)): #if b< probability calculated that a photon reach the detector 0
        M=0 #we define that the photon reach the detector 0
    else: # else the photon reach the detector 1
        M=1
        
    if M==a: # if the photon reach the same detector where the agent made the bet 
        return (w) #the agent receive a reward
    else :
        return(l)#else the agent lose
    

In [3]:
class BanditAlgorithm:
    def __init__(self,N_actions,eps,steps,environment):
        self.N_actions=N_actions
        self.eps=eps
        self.steps=steps
        self.environment=environment
    def run(self):
        Q=np.zeros(self.N_actions)
        N=np.zeros(self.N_actions)
        for x in range(self.steps):
            if random.random() < 1-self.eps:
                a=np.argmax(Q)
            else:
                a=random.randint(0,(self.N_actions-1))
            R=self.environment(a)
            N[a]=N[a]+1
            Q[a]=Q[a]+(R-Q[a])/N[a]
        return Q
         
            
            

In [4]:
multibandit=BanditAlgorithm(2,0.1,10000,basicbandit)
Q=multibandit.run()
for y in range(10):
    print('episode {}'.format(y))
    print('Q value:')
    print(Q)
    print('Best action to choose: {}'.format(np.argmax(Q)))
    print('**********')

episode 0
Q value:
[0.8487015 0.1595092]
Best action to choose: 0
**********
episode 1
Q value:
[0.8487015 0.1595092]
Best action to choose: 0
**********
episode 2
Q value:
[0.8487015 0.1595092]
Best action to choose: 0
**********
episode 3
Q value:
[0.8487015 0.1595092]
Best action to choose: 0
**********
episode 4
Q value:
[0.8487015 0.1595092]
Best action to choose: 0
**********
episode 5
Q value:
[0.8487015 0.1595092]
Best action to choose: 0
**********
episode 6
Q value:
[0.8487015 0.1595092]
Best action to choose: 0
**********
episode 7
Q value:
[0.8487015 0.1595092]
Best action to choose: 0
**********
episode 8
Q value:
[0.8487015 0.1595092]
Best action to choose: 0
**********
episode 9
Q value:
[0.8487015 0.1595092]
Best action to choose: 0
**********


In [20]:
def Drop_a_photon():
    fi=math.pi/4
    c=random.random()
    if c<0.5*(1+math.cos(fi)):
        return(0)
    else:
        return(1)

In [6]:
class Environment_Photon:
    def __init__(self,N_actions):
        self.observations_space=3
        self.action_space=N_actions
        self.state=0
        self.memory=0
        self.boolean=False
        
    def step(self,a):
        if self.state==0:
            #print('compra')
            click=Drop_a_photon()
            self.memory=click
            act=self.action_space/2
            if a<act:
                beat=0
                u=2.0*a/(act-1)
            else:
                beat=1
                u=(a-act)*2.0/(act-1)
                
            if click == beat:
                reward=1-u
            else:
                reward=-u
            self.state=1
            self.boolean=False
            #print('Reward')
            #print(reward)
            #print('action')
            #print(a)
            #print('prob')
            #print(u)
            return self,reward,self.boolean
        elif self.state==1:
            #print('venta')
            click=self.memory
            act=self.action_space/2
            if a<act:
                beat=0
                u=2.0*a/(act-1)
            else:
                beat=1
                u=(a-act)*2.0/(act-1)
            
            if click==beat:
                reward= u-1
            else:
                reward=u
            self.state=2
            self.boolean=True
            #print('Reward')
            #print(reward)
            #print('action')
            #print(a)
            #print('prob')
            #print(u)
            #print('***************')
            #print('***************')
            return self,reward,self.boolean
        else:
            #print('Final state')
            self.state=3
            reward=0
            self.boolean=True
            return self,reward,self.boolean
    
    def reset(self):
        #print("reseteado")
        self.state=0
        self.boolean=False
        return self
                
        

In [13]:
class QTable:
    def __init__(self,LearningRate,Eps,environment,maxsteps,discount,Number_episodes):
        self.lr=LearningRate
        self.eps=Eps
        self.environment=environment
        self.maxsteps=maxsteps
        self.discount=discount
        self.Number_episodes=Number_episodes
        
    def run(self):
        Q=np.zeros((self.environment.observations_space,self.environment.action_space))
        for i in range(self.Number_episodes):
            
            s=self.environment.reset()
            d=False
            j=0
        
        
            while j < self.maxsteps:
                j+=1
                if random.random() < 1-self.eps:
                    a=np.argmax(Q[s.state,:])
                else:
                    a=random.randint(0,(self.environment.action_space-1))
                aux=s.state
                s1,r,d = self.environment.step(a)
                Q[aux,a] = Q[aux,a] + self.lr*(r + self.discount*np.max(Q[s1.state,:]) - Q[aux,a])
                s=s1
                if d==True:
                    break
            
        return Q
            

In [14]:
N_actions=20
env=Environment_Photon(N_actions)
rl=QTable(0.8,0.1,env,99,0.9,1000)
aux=N_actions/2
for i in range(20):
    Q=rl.run()
    h1=np.argmax(Q[0,:]) 
    h2=np.argmax(Q[1,:])
    if h1 >= aux:
        u=(h1-aux)*2.0/(aux-1)
        det1=1
    else:
        u=h1* 2.0/(aux-1)
        det1=0
    if h2 >= aux:
        v=(h2-aux)*2.0/(aux-1)
        det2=1
    else:
        v=h2* 2.0/(aux-1)
        det2=0
            
    print('episodio %i' %i)
    print('argmax')
    print(h1)
    print(h2)
    print('compra detector %d' %det1)    
    print(u)
    print('venta detector %d ' %det2)
    print(v)
    print('*************')

episodio 0
argmax
0
19
compra detector 0
0.0
venta detector 1 
2.0
*************
episodio 1
argmax
0
19
compra detector 0
0.0
venta detector 1 
2.0
*************
episodio 2
argmax
0
19
compra detector 0
0.0
venta detector 1 
2.0
*************
episodio 3
argmax
0
19
compra detector 0
0.0
venta detector 1 
2.0
*************
episodio 4
argmax
0
19
compra detector 0
0.0
venta detector 1 
2.0
*************
episodio 5
argmax
0
19
compra detector 0
0.0
venta detector 1 
2.0
*************
episodio 6
argmax
0
19
compra detector 0
0.0
venta detector 1 
2.0
*************
episodio 7
argmax
0
19
compra detector 0
0.0
venta detector 1 
2.0
*************
episodio 8
argmax
0
19
compra detector 0
0.0
venta detector 1 
2.0
*************
episodio 9
argmax
0
19
compra detector 0
0.0
venta detector 1 
2.0
*************
episodio 10
argmax
0
19
compra detector 0
0.0
venta detector 1 
2.0
*************
episodio 11
argmax
0
19
compra detector 0
0.0
venta detector 1 
2.0
*************
episodio 12
argmax
0
19
co

In [15]:
class SARSA:
    def __init__(self,LearningRate,Eps,environment,maxsteps,discount,Number_episodes):
        self.lr=LearningRate
        self.eps=Eps
        self.environment=environment
        self.maxsteps=maxsteps
        self.discount=discount
        self.Number_episodes=Number_episodes
        
    def run(self):
        Q=np.zeros((self.environment.observations_space,self.environment.action_space))
        for i in range(self.Number_episodes):
            
            s=self.environment.reset()
            d=False
            j=0
        
        
            while j < self.maxsteps:
                j+=1
                if random.random() < 1-self.eps:
                    a0=np.argmax(Q[s.state,:])
                else:
                    a0=random.randint(0,(self.environment.action_space-1))
                aux=s.state
                s1,r,d = self.environment.step(a0)
                if random.random() < 1-self.eps:
                    a1=np.argmax(Q[s1.state,:])
                else:
                    a1=random.randint(0,(self.environment.action_space-1))
                    
                Q[aux,a0] = Q[aux,a0] + self.lr*(r + self.discount*Q[s1.state,a1] - Q[aux,a0])
                s=s1
                if d==True:
                    break
            
        return Q

In [19]:
N_actions=20
env=Environment_Photon(N_actions)
rl=SARSA(0.8,0.1,env,99,0.3,10000)
aux=N_actions/2
for i in range(20):
    Q=rl.run()
    h1=np.argmax(Q[0,:]) 
    h2=np.argmax(Q[1,:])
    if h1 >= aux:
        u=(h1-aux)*2.0/(aux-1)
        det1=1
    else:
        u=h1* 2.0/(aux-1)
        det1=0
    if h2 >= aux:
        v=(h2-aux)*2.0/(aux-1)
        det2=1
    else:
        v=h2* 2.0/(aux-1)
        det2=0
            
    print('episodio %i' %i)
    print('argmax')
    print(h1)
    print(h2)
    print('compra detector %d' %det1)    
    print(u)
    print('venta detector %d ' %det2)
    print(v)
    print('*************')

episodio 0
argmax
0
19
compra detector 0
0.0
venta detector 1 
2.0
*************
episodio 1
argmax
0
19
compra detector 0
0.0
venta detector 1 
2.0
*************
episodio 2
argmax
0
19
compra detector 0
0.0
venta detector 1 
2.0
*************
episodio 3
argmax
0
19
compra detector 0
0.0
venta detector 1 
2.0
*************
episodio 4
argmax
0
19
compra detector 0
0.0
venta detector 1 
2.0
*************
episodio 5
argmax
0
19
compra detector 0
0.0
venta detector 1 
2.0
*************
episodio 6
argmax
1
19
compra detector 0
0.2222222222222222
venta detector 1 
2.0
*************
episodio 7
argmax
0
19
compra detector 0
0.0
venta detector 1 
2.0
*************
episodio 8
argmax
0
19
compra detector 0
0.0
venta detector 1 
2.0
*************
episodio 9
argmax
0
19
compra detector 0
0.0
venta detector 1 
2.0
*************
episodio 10
argmax
1
19
compra detector 0
0.2222222222222222
venta detector 1 
2.0
*************
episodio 11
argmax
1
19
compra detector 0
0.2222222222222222
venta detector 1 

In [21]:
class Water_FoodEnvironment:
    def __init__(self,N_actions):
        self.N_actions=N_actions
        
    def run(self,a):
        act=self.N_actions/2
        if a<act:
            beat=0
            u=2.0*a/(act-1)
        else:
            beat=1
            u=(a-act)*2.0/(act-1)
        
        click1=Drop_a_photon()
        if beat==click1:
            water=1-u
        else:
            water=-u
            
        click2=Drop_a_photon()
        if beat==click2:
            food=u-1
        else:
            food=u
        
        reward=min(water,food)
        
        return reward
        

In [25]:
class BanditAlgorithm_WhitOptimisticValues:
    def __init__(self,N_actions,eps,steps,environment,optimisticvalues):
        self.N_actions=N_actions
        self.eps=eps
        self.steps=steps
        self.environment=environment
        self.optimistic_values=optimisticvalues
    def run(self):
        Q=np.empty(self.N_actions)
        Q.fill(self.optimistic_values)
        N=np.zeros(self.N_actions)
        for x in range(self.steps):
            if random.random() < 1-self.eps:
                a=np.argmax(Q)
            else:
                a=random.randint(0,(self.N_actions-1))
            R=self.environment.run(a)
            N[a]=N[a]+1
            Q[a]=Q[a]+(R-Q[a])/N[a]
        return Q
         
            

In [31]:
TotalActions=20
aux=TotalActions/2
env=Water_FoodEnvironment(TotalActions)
multibandit=BanditAlgorithm_WhitOptimisticValues(TotalActions,0.1,10000,env,10)
Q=multibandit.run()
for y in range(10):
    Q=multibandit.run()
    h1=np.argmax(Q)
    
    print('episode {}'.format(y))
    #print('Q value:')
    print(Q)
    print('Best action to choose: {}'.format(np.argmax(Q)))
    if h1 >= aux:
        u=(h1-aux)*2.0/(aux-1)
        det1=1
    else:
        u=h1* 2.0/(aux-1)
        det1=0
    print('u value choose')
    print(u)
    print('**********')

episode 0
[-0.94117647 -0.62962963 -0.37333333 -0.27350427 -0.18106996 -0.24904215
 -0.51666667 -0.80555556 -0.93295019 -1.11111111 -0.14549271 -0.23434343
 -0.34188034 -0.58490566 -0.78888889 -0.94832041 -1.08805031 -1.41269841
 -1.50694444 -1.83673469]
Best action to choose: 10
u value choose
0.0
**********
episode 1
[-0.77777778 -0.65277778 -0.47435897 -0.25581395 -0.16548463 -0.33333333
 -0.43502825 -0.6984127  -0.83232323 -1.20338983 -0.14677473 -0.23958333
 -0.28976035 -0.52469136 -0.73879142 -0.93333333 -1.23611111 -1.46296296
 -1.61706349 -1.86486486]
Best action to choose: 10
u value choose
0.0
**********
episode 2
[-0.8627451  -0.68085106 -0.47222222 -0.28828829 -0.19039077 -0.32850242
 -0.62910798 -0.75555556 -0.93567251 -1.04255319 -0.12389381 -0.28205128
 -0.30359937 -0.54761905 -0.74213836 -0.96476965 -1.13768116 -1.43055556
 -1.5821256  -1.83333333]
Best action to choose: 10
u value choose
0.0
**********
episode 3
[-0.89473684 -0.54320988 -0.33127572 -0.2654321  -0.19541